In [1]:
import pandas as pd
data = pd.read_csv("dữ liệu chatbot question-answer short style.csv")
data
    

,Unnamed: 0,user_a,user_b
0,0,Thích mẫu người nào,"Dễ thương, tóc dài, da trắng"
1,1,Có crush ai không,Có 1 bạn cùng lớp
2,2,Tại sao lại thích bạn dó,Vì đáp ứng những yêu cầu của tao
3,3,Có hay nói chuyện không,Hay nhắn tin messenger
4,4,Bạn kia có bắt chuyện trước không,Có đôi khi
...,...,...,...
5895,5895,Nhà mày bao nhiêu người?,7 8 người gì đó
5896,5896,Có bao nhiêu anh chị em?,3 anh chị em
5897,5897,Mày là em út hay anh cả?,Anh út
5898,5898,Bố mày làm nghề gì?,chủ tịch giả nghèo và cái kết


In [2]:
user_a = data.loc[:,"user_a"]
user_a

0                     Thích mẫu người nào
1                       Có crush ai không
2                Tại sao lại thích bạn dó
3                 Có hay nói chuyện không
4       Bạn kia có bắt chuyện trước không
                      ...                
5895             Nhà mày bao nhiêu người?
5896             Có bao nhiêu anh chị em?
5897             Mày là em út hay anh cả?
5898                  Bố mày làm nghề gì?
5899                   Chị mày làm ở đâu?
Name: user_a, Length: 5900, dtype: object

In [3]:
user_b = data.loc[:,"user_b"]
user_b

0           Dễ thương, tóc dài, da trắng
1                      Có 1 bạn cùng lớp
2       Vì đáp ứng những yêu cầu của tao
3                 Hay nhắn tin messenger
4                             Có đôi khi
                      ...               
5895                     7 8 người gì đó
5896                        3 anh chị em
5897                              Anh út
5898       chủ tịch giả nghèo và cái kết
5899                      Tao mồ côi chị
Name: user_b, Length: 5900, dtype: object

In [4]:
# zipping pair for a and b
pairs = list(zip(user_a,user_b))
pairs

[('Thích mẫu người nào', 'Dễ thương, tóc dài, da trắng'),
 ('Có crush ai không', 'Có 1 bạn cùng lớp'),
 ('Tại sao lại thích bạn dó', 'Vì đáp ứng những yêu cầu của tao'),
 ('Có hay nói chuyện không', 'Hay nhắn tin messenger'),
 ('Bạn kia có bắt chuyện trước không', 'Có đôi khi'),
 ('Có định rủ bạn đó về quê dịp Tết không', 'Có, đang đợi trả lời'),
 ('Có hay đi chơi với nhau không', 'Thường đi cùng cả nhóm bạn'),
 ('Sở thích mày sao?', 'Thích phim, đồ ăn, thể thao'),
 ('Thích phim gì?', 'Phim ma'),
 ('Thích chơi gì?', 'chơi đá banh'),
 ('Thích ăn gì?', 'Ăn cơm sườn'),
 ('mày thích chơi bi lắc không?', 'có'),
 ('mày chơi giỏi nhất cây nào?', 'một, năm'),
 ('mày chơi cầu công hay cầu thủ?', 'thủ'),
 ('có chơi cầu lông không?', 'Không thường xuyên'),
 ('hay chơi với ai?', 'Đồng nghiệp'),
 ('chơi ở đâu?', 'quận 10'),
 ('chơi bao nhiêu lần một tuần?', '1 lần'),
 ('chơi sân nào?', 'sân kỳ hòa'),
 ('thích phim nào?', 'quái vật'),
 ('thích diễn viên nào không?', 'không có'),
 ('thích hãng phim n

In [10]:
import numpy as np
import re
input_docs = []
target_docs = []
input_tokens = set()
target_tokens = set()
for line in pairs[:200]:
  input_doc, target_doc = line[0], line[1]
  # Nối mỗi câu đầu vào vào input_docs
  input_docs.append(input_doc)
  # Tách từ khỏi dấu câu  
  target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc))
  # Xác định lại target_doc bên dưới và nối nó vào target_docs
  target_doc = '<START> ' + target_doc + ' <END>'
  target_docs.append(target_doc)
  
  # Chia từng câu thành các từ và thêm từng từ duy nhất vào bộ từ vựng
  for token in re.findall(r"[\w']+|[^\s\w]", input_doc):
    if token not in input_tokens:
      input_tokens.add(token)
  for token in target_doc.split():
    if token not in target_tokens:
      target_tokens.add(token)
input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

In [ ]:
#input_docs

In [ ]:
#target_docs

In [11]:
input_features_dict = dict(
    [(token, i) for i, token in enumerate(input_tokens)])
target_features_dict = dict(
    [(token, i) for i, token in enumerate(target_tokens)])
reverse_input_features_dict = dict(
    (i, token) for token, i in input_features_dict.items())
reverse_target_features_dict = dict(
    (i, token) for token, i in target_features_dict.items())

Training

In [12]:
#Maximum length of sentences in input and target documents
max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])
max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])
encoder_input_data = np.zeros(
    (len(input_docs), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
for line, (input_doc, target_doc) in enumerate(zip(input_docs, target_docs)):
    for timestep, token in enumerate(re.findall(r"[\w']+|[^\s\w]", input_doc)):
        #Assign 1. for the current line, timestep, & word in encoder_input_data
        encoder_input_data[line, timestep, input_features_dict[token]] = 1.
    
    for timestep, token in enumerate(target_doc.split()):
        decoder_input_data[line, timestep, target_features_dict[token]] = 1.
        if timestep > 0:
            decoder_target_data[line, timestep - 1, target_features_dict[token]] = 1.

Encoder-decoder training and LSTM

In [13]:
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model
#Dimensionality
dimensionality = 256
#The batch size and number of epochs
batch_size = 10
epochs = 600
#Encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(dimensionality, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]
#Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(dimensionality, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
#Model
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
#Compiling
training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')
#Training
training_model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)
training_model.save('training_model.h5')

Epoch 1/600
16/16 [==============================] - 16s 216ms/step - loss: 0.9614 - accuracy: 0.0402 - val_loss: 0.5308 - val_accuracy: 0.0435
Epoch 2/600
16/16 [==============================] - 2s 131ms/step - loss: 0.8930 - accuracy: 0.0435 - val_loss: 0.5290 - val_accuracy: 0.0435
Epoch 3/600
16/16 [==============================] - 2s 138ms/step - loss: 0.8831 - accuracy: 0.0437 - val_loss: 0.5342 - val_accuracy: 0.0435
Epoch 4/600
16/16 [==============================] - 2s 134ms/step - loss: 0.8750 - accuracy: 0.0465 - val_loss: 0.5365 - val_accuracy: 0.0446
Epoch 5/600
16/16 [==============================] - 2s 131ms/step - loss: 0.8693 - accuracy: 0.0470 - val_loss: 0.5328 - val_accuracy: 0.0478
Epoch 6/600
16/16 [==============================] - 2s 132ms/step - loss: 0.8665 - accuracy: 0.0473 - val_loss: 0.5359 - val_accuracy: 0.0478
Epoch 7/600
16/16 [==============================] - 2s 131ms/step - loss: 0.8624 - accuracy: 0.0476 - val_loss: 0.5373 - val_accuracy: 0.047

In [19]:
# Chạy Chatbot
class ChatBot:
  negative_responses = ("no", "nope", "nah", "naw", "not a chance", "sorry")
  exit_commands = ("quit", "pause", "exit", "goodbye", "bye", "later", "stop")
#Trò chuyện mở đầu
  def start_chat(self):
    user_response = input("Hi, I'm a chatbot trained on random dialogs. Would you like to chat with me?\n")
    
    if user_response in self.negative_responses:
      print("Ok, have a great day!")
      return
    self.chat(user_response)
#Xử lý trò chuyện
  def chat(self, reply):
    while not self.make_exit(reply):
      reply = input(self.generate_response(reply)+"\n")
    
  #Chuyển đổi đầu vào thành ma trận
  def string_to_matrix(self, user_input):
    tokens = re.findall(r"[\w']+|[^\s\w]", user_input)
    user_input_matrix = np.zeros(
      (1, max_encoder_seq_length, num_encoder_tokens),
      dtype='float32')
    for timestep, token in enumerate(tokens):
      if token in input_features_dict:
        user_input_matrix[0, timestep, input_features_dict[token]] = 1.
    return user_input_matrix
  
  #Tạo phản hồi bằng model sequence2sequence
  def generate_response(self, user_input):
    input_matrix = self.string_to_matrix(user_input)
    chatbot_response = decode_response(input_matrix)
    #Xóa mã thông báo <START> và <END> khỏi chatbot_response
    chatbot_response = chatbot_response.replace("<START>",'')
    chatbot_response = chatbot_response.replace("<END>",'')
    return chatbot_response
#kiểm tra các lệnh thoát
  def make_exit(self, reply):
    for exit_command in self.exit_commands:
      if exit_command in reply:
        print("Ok, have a great day!")
        return True
    return False
  
chatbot = ChatBot()

In [22]:
chatbot.start_chat()


Hi, I'm a chatbot trained on random dialogs. Would you like to chat with me?
no
Ok, have a great day!
